# Using the GM Data Scala SDK inside Polynote<br>



## Config



<div>This is a configuration object used by the rest of the apps in this tutorial.It's members can be accessed like `Config.security` or it can be imported to use the variables directly with `import Config._`s<br></div>

In [2]:
import io.circe.parser.parse
import io.circe.parser.decode
import io.circe.generic.auto._
import io.greymatter.gmdata.sdk._
import org.http4s.{Headers, Header, Uri}
object Config{
    val security = Some(Security("UNCLASSIFIED", "#FFFFFF", "#007A33"))
    val objectPolicy = Some(parse("""{"label":"fullAccess","requirements":{"f":"owner-full-ro-all","a":[{"v":"userDN"},{"v":"cn=seth.lasky,ou=di2e,o=u.s. government,l=alexandria,st=virginia,c=us"}]}}""").right.get)
    val originalObjectPolicy = Some("""(if (contains userDN "cn=seth.lasky,ou=di2e,o=u.s. government,l=alexandria,st=virginia,c=us")(yield-all)(yield R X))""")
    val rootUrl = Uri.unsafeFromString("https://mesh.greymatter.devcloud.di2e.net/services/drive-data-113/latest")
    val headers = Headers(List())
}

In [3]:
import java.io.{File, FileInputStream, FileOutputStream}
import javax.net.ssl.{KeyManagerFactory, SSLContext, TrustManagerFactory}
import java.security.KeyStore
import fs2.text._
import fs2.Stream
import cats.effect.IO
trait SSL {
    def createSslContext(keyStoreFile: File,
                       keyStorePassword: String,
                       trustStoreFile: File,
                       trustStorePassword: String): SSLContext = {

        val keyStoreStream = new FileInputStream(keyStoreFile)
        val trustStoreStream = new FileInputStream(trustStoreFile)
        val keyManagerFactory = KeyManagerFactory.getInstance(KeyManagerFactory.getDefaultAlgorithm)
        val trustManagerFactory = TrustManagerFactory.getInstance(KeyManagerFactory.getDefaultAlgorithm)
        val keyStore = KeyStore.getInstance(KeyStore.getDefaultType)
        val trustStore = KeyStore.getInstance(KeyStore.getDefaultType)
        val keyPass = keyStorePassword.toCharArray
        val trustPass = trustStorePassword.toCharArray
        keyStore.load(keyStoreStream, keyPass)
        keyManagerFactory.init(keyStore, keyPass)
        trustStore.load(trustStoreStream, trustPass)
        trustManagerFactory.init(trustStore)
        val sslContext = SSLContext.getInstance("TLS")
        sslContext.init(keyManagerFactory.getKeyManagers, trustManagerFactory.getTrustManagers, null)
        sslContext
    }

    lazy val keyFile = new File("/Users/sethlasky/certstuff/sethcert.p12")
    lazy val trustFile =  new File("/Users/sethlasky/certstuff/trust.p12")
    lazy val sslContext = Some(createSslContext(keyFile, "password", trustFile, "password"))
}

## Writing to GM Data



### UserFolderClient

<div>Contains a convenience function for getting the metadata of the userfield folder in GM Data or creating it if it doesn't exist. There is also a method for getting the oid of a metadata object.<br></div>

In [6]:
import io.greymatter.gmdata.sdk.GmData
import cats.effect.IO
import org.http4s.client.Client
import scala.concurrent.ExecutionContext
import cats.effect.ContextShift
import cats.effect.Timer
import fs2.Stream
import io.circe.Json
import cats.implicits._


trait UserFolderClient extends GmData[IO]{
    import Config._
    def userFolder(client: Client[IO]) = getOrCreateUserFolder(rootUrl, headers, client, None, security, originalObjectPolicy)

    def getOid(metadata: Either[Throwable, Metadata]) = metadata.map(_.oid.get) match {
        case Right(oid) => oid
        case Left(err) => throw new Throwable(s"Couldn't get user folder oid: $err")
    }
}



### WriteToGmDataClient

<div>Contains functions for writing very specific things to GMData. It supports writing folders, specific text files with a random color as the contents and writing image files. There are also convenience methods for creating metadata objects.<br></div>

In [8]:
import java.time.LocalDateTime
import scala.util.Random

trait WriteToGmDataClient extends GmData[IO]{
    import Config._

    def createTextFileStream(str: String) = Stream.emits(str.getBytes)

    def createMetadataForFile(parentOid: String, name: String, mimetype: Option[String] = Some("text/plain")) = 
        Metadata(parentoid = parentOid, 
        name = name, 
        objectpolicy = objectPolicy, 
        mimetype = mimetype, 
        size = None, 
        action = "C", 
        security = security, 
        None, 
        None, 
        None, 
        None, 
        None, 
        None)
    

    def createMetadataForFolder(parentOid: String, name: String) = 
        Metadata(parentoid = parentOid, 
        name = name, 
        objectpolicy = objectPolicy, 
        mimetype = None,
        size = None, 
        action = "C", 
        security = security, 
        originalObjectPolicy = None, 
        custom = None, 
        oid = None, 
        tstamp = None, 
        policy = None,
        isdir = Some(true),
        isfile = None, allowPartialPermissions = None, sha256plain = None)
    
    case class ColorFile(metadata: List[Metadata], file: Stream[IO, Byte])

    def createColorFile(parentOid: String, name: String) = {
        val colors = List("Greys", "Reds", "Oranges", "ocean", "magma", "rainbow", "coolwarm", "copper")
        val fileContents = colors(new Random().nextInt(colors.length))
        val fileStream = createTextFileStream(fileContents)
        val metadata = createMetadataForFile(parentOid, name)
        ColorFile(List(metadata), fileStream)
    }
    def writeColorFile(parentOid: String, name: String, client: Client[IO]) = {
        val colorFile = createColorFile(parentOid, name)
        createFile(colorFile.metadata, rootUrl, headers, colorFile.file, client)        
    }

    def writeLotsOfColorFiles(parentOid: String, numberOfFiles: Int, client: Client[IO]) = {        
        val timestamp = LocalDateTime.now.toString
        val streamOfFileNames = Stream.emit(1).repeatN(numberOfFiles).scan1(_ + _).map(fileNumber => s"Random color $fileNumber $timestamp")
        Stream.eval(writeFolder(parentOid, timestamp, client)).flatMap{ folderMetadata =>
            streamOfFileNames.evalMap(writeColorFile(folderMetadata.oid.get, _, client))
        }
    }   

    def writeFolder(parentOid: String, name: String, client: Client[IO]) = {
        val metadata = createMetadataForFolder(parentOid, name)
        createFolder(List(metadata), rootUrl, headers, client).map(_.head)
    } 

    def writeImageFile(parentOid: String, name: String, client: Client[IO], image: Stream[IO, Byte]) = {
        val metadata = createMetadataForFile(parentOid, name, Some("image/jpeg"))
        createFile(List(metadata), rootUrl, headers, image, client)        
    }

    def writeAppendable(parentOid: String, name: String, client: Client[IO], chunk: Option[Int], file: Stream[IO, Byte], metadata: Metadata) = {
        createAppendableFile(metadata, file, rootUrl, headers, client, chunk).reduce((l,c) => c)
    }

    def appendFile(metadata: Metadata, file: Stream[IO, Byte], client: Client[IO]) = {
        append(metadata, file, rootUrl, headers, client)
    }

    
}

### SimpleWritingApp

<div>Now that we have some traits written for convenience, let's extend them from an application and use their methods along with the SDK.&nbsp; Since the traits already extend the SDK we don't have to from the app. This app does the following:</div><div><ol><li>Gets the default client</li><li>Gets the userfield folder oid <br></li><li>Writes a number of text files to GM Data with the contents being a random color mapping.</li><li>Gets the value of those file's metadata from the responses of the write operations</li><li>Saves metadata list to the variable `metadataList`<br></li></ol></div>

In [10]:
import java.time.LocalDateTime
object SimpleWritingApp extends UserFolderClient with WriteToGmDataClient with SSL {
    implicit lazy val ec = ExecutionContext.global
    implicit val ctxShift: ContextShift[IO] = IO.contextShift(ec)
    implicit val timer: Timer[IO] = IO.timer(ec)

    val numberOfFiles = 20

    val timestamp = LocalDateTime.now.toString

    val mainFolder = for{
        client <- defaultClient(sslContext).stream
        userFolderMetadata <- Stream.eval(userFolder(client))
        userFolderOid = getOid(userFolderMetadata)
        folder <- Stream.eval(writeFolder(userFolderOid, timestamp + "newFolder", client))
    } yield folder.oid.get

    def run(folderOid: String) = for{
        client <- defaultClient(sslContext).stream
        metadatas <- writeLotsOfColorFiles(folderOid, numberOfFiles, client)
    } yield metadatas
}
val mainFolderOid = SimpleWritingApp.mainFolder.compile.toList.unsafeRunSync.head
val metadataList = SimpleWritingApp.run(mainFolderOid).compile.toList.unsafeRunSync().flatten

## Plotting data natively in Polynote

<div>Polynote has hooks for primitive data types to be used in a few graphs. The metadata object returned has a few fields that can't be mapped natively. Plottable is a case class we're mapping to here in order for Polynote to be able to display some sort of graph using our data info.<br></div>

In [13]:
case class Plottable(name: String, secondsSinceUploaded: Long, size: Long) 
lazy val plottable = (metadata: Metadata) => { 
    val now = System.currentTimeMillis 
    val tstamp = java.lang.Long.parseLong(metadata.tstamp.get, 16) / 1000000 
    val secondsSinceUploaded = (now - tstamp) / 1000 
    val size = metadata.size.get 
    Plottable(metadata.name, secondsSinceUploaded, size) 
}

metadataList map plottable

List(Plottable(Random color 1 2021-03-09T11:22:35.523,5,6), Plottable(Random color 2 2021-03-09T11:22:35.523,5,5), Plottable(Random color 3 2021-03-09T11:22:35.523,5,8), Plottable(Random color 4 2021-03-09T11:22:35.523,4,8), Plottable(Random color 5 2021

## Reading from GM Data<br>



### ReadFromGmDataClient

<div>Contains convenience functions for getting all files and folders from a path recursively. Also contains functions for streaming files.<br></div>

In [16]:
import java.time.LocalDateTime
import cats.implicits._
import fs2.text._

trait ReadFromGmDataClient extends GmData[IO]{
    import Config._

    def listAllObjectsFromPath(path: String, client: Client[IO]) = getFileList(path, headers, rootUrl, client)

    def listAllFilesFromPath(path: String, client: Client[IO], recursive: Boolean = false): IO[List[Metadata]] = for{
        filesAndFolders <- listAllObjectsFromPath(path, client)
        files <- filesAndFolders.flatTraverse {
            case metadata if metadata.isfile == Some(true) => IO(List(metadata))
            case folderMetadata if recursive => listAllFilesFromPath(folderMetadata.oid.get, client, recursive)
            case _ => IO(List())
        }            
     } yield files

    def getFile(path: String, client: Client[IO]) = streamFile(path, headers, rootUrl, client)    

    def getTextFile(path: String, client: Client[IO]) = getFile(path, client).through(utf8Decode)

    def getImageFile(path: String, client: Client[IO]) = getFile(path, client).through(base64Encode)
}

### SimpleReadingApp

<div>We have a client written for listing and reading files, now lets write a small application. This application does the following:</div><div><ol><li>Gets a default client</li><li>Gets the oid from the userfield folder</li><li>Recursively lists all metadata from all files currently under that folder</li><li>Streams over that list and filters by metadata where mimetype is "text/plain"</li><li>Gets the contents of those files and returns a list of text, in this case our previous app uploaded files with colors in them, so this returns a list of colors.</li><li>Saves that list to the variable colorsList<br></li></ol></div>

In [18]:
import java.time.LocalDateTime
import cats.implicits._
object SimpleReadingApp extends UserFolderClient with ReadFromGmDataClient with SSL {
    implicit lazy val ec = ExecutionContext.global
    implicit val ctxShift: ContextShift[IO] = IO.contextShift(ec)
    implicit val timer: Timer[IO] = IO.timer(ec)

    def run(folderOid: String) = for {
        client <- defaultClient(sslContext).stream
        fileList <- Stream.eval(listAllFilesFromPath(folderOid, client, true))
        textFileMetadata <- Stream.emits(fileList).filter(_.mimetype == Some("text/plain"))
        colors <- getTextFile(getOid(Right(textFileMetadata)), client)
    } yield colors
}

val colorsList = SimpleReadingApp.run(mainFolderOid).compile.toList.unsafeRunSync()


## Interopping from Scala to Python



### Converting Scala objects for use in Python

<div>In order for Polynote to allow Python and Scala programs to interop there is a conversion between some primitive types using Jep. For more info and the full list of Python -&gt; Scala mappings go to https://github.com/ninia/jep/wiki/How-Jep-Works. In this field we turn the list of colors into an array from a list so it can be consumed by Python libraries<br></div>

In [21]:
val colors = colorsList.toArray

### Simple Histogram Plot using matplotlib

<div>This is a quick plot that shows the number of elements in the list with a specific color name, uses the `colors` variable from the last cell.<br></div>

In [23]:
import matplotlib.pyplot as plt
import numpy as np
plt.hist(colors)
plt.show()


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 2021-03-09T11:22:47.152218 
 image/svg+xml 
 
 
 Matplotlib v3.3.4, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- copper -->
 
 
 <path d="M 48.78125 52.59375 
L 48.78125 44.1875 
Q 44.96875 46.296875 41.140625 47.34375 
Q 37.3125 48.390625 33.40625 48.390625 
Q 24.65625 48.390625 19.8125 42.84375 
Q 14.984375 37.3125 14.984375 27.296875 
Q 14.984375 17.28125 19.8125 11.734375 
Q 24.65625 6.203125 33.40625 6.203125 
Q 37.3125 6.203125 41.140625 7.25 
Q 44.96875 8.296875 48.78125 10.40625 
L 48.78125 2.09375 
Q 45.015625 0.34375 40.984375 -0.53125 
Q 36.96875 -1.421875 32.421875 -1.421875 
Q 20.0625 -1.421875 12.78125 6.34375 
Q 5.515625 14.109375 5.515625 27.296875 
Q 5.515625 40.671875 12.859375 48.328125 
Q 20.21875 56 33.015625 56 
Q 37.15625 56 41.109375 55.140625 
Q 45.0625 54.296875 48.78125 52.59375 
z
" id="DejaVuSans-99"/>
 <path d="M 30.609375 48.390625 
Q 23.390625 48.390625 19.1875 42.75 
Q 14.984375 37.109375 14.984375 27.296875 
Q 14.984375 17.484375 19.15625 11.84375 
Q 23.34375 6.203125 30.609375 6.203125 
Q 37.796875 6.203125 41.984375 11.859375 
Q 46.1875 17.53125 46.1875 27.296875 
Q 46.1875 37.015625 41.984375 42.703125 
Q 37.796875 48.390625 30.609375 48.390625 
z
M 30.609375 56 
Q 42.328125 56 49.015625 48.375 
Q 55.71875 40.765625 55.71875 27.296875 
Q 55.71875 13.875 49.015625 6.21875 
Q 42.328125 -1.421875 30.609375 -1.421875 
Q 18.84375 -1.421875 12.171875 6.21875 
Q 5.515625 13.875 5.515625 27.296875 
Q 5.515625 40.765625 12.171875 48.375 
Q 18.84375 56 30.609375 56 
z
" id="DejaVuSans-111"/>
 <path d="M 18.109375 8.203125 
L 18.109375 -20.796875 
L 9.078125 -20.796875 
L 9.078125 54.6875 
L 18.109375 54.6875 
L 18.109375 46.390625 
Q 20.953125 51.265625 25.265625 53.625 
Q 29.59375 56 35.59375 56 
Q 45.5625 56 51.78125 48.09375 
Q 58.015625 40.1875 58.015625 27.296875 
Q 58.015625 14.40625 51.78125 6.484375 
Q 45.5625 -1.421875 35.59375 -1.421875 
Q 29.59375 -1.421875 25.265625 0.953125 
Q 20.953125 3.328125 18.109375 8.203125 
z
M 48.6875 27.296875 
Q 48.6875 37.203125 44.609375 42.84375 
Q 40.53125 48.484375 33.40625 48.484375 
Q 26.265625 48.484375 22.1875 42.84375 
Q 18.109375 37.203125 18.109375 27.296875 
Q 18.109375 17.390625 22.1875 11.75 
Q 26.265625 6.109375 33.40625 6.109375 
Q 40.53125 6.109375 44.609375 11.75 
Q 48.6875 17.390625 48.6875 27.296875 
z
" id="DejaVuSans-112"/>
 <path d="M 56.203125 29.59375 
L 56.203125 25.203125 
L 14.890625 25.203125 
Q 15.484375 15.921875 20.484375 11.0625 
Q 25.484375 6.203125 34.421875 6.203125 
Q 39.59375 6.203125 44.453125 7.46875 
Q 49.3125 8.734375 54.109375 11.28125 
L 54.109375 2.78125 
Q 49.265625 0.734375 44.1875 -0.34375 
Q 39.109375 -1.421875 33.890625 -1.421875 
Q 20.796875 -1.421875 13.15625 6.1875 
Q 5.515625 13.8125 5.515625 26.8125 
Q 5.515625 40.234375 12.765625 48.109375 
Q 20.015625 56 32.328125 56 
Q 43.359375 56 49.78125 48.890625 
Q 56.203125 41.796875 56.203125 29.59375 
z
M 47.21875 32.234375 
Q 47.125 39.59375 43.09375 43.984375 
Q 39.0625 48.390625 32.421875 48.390625 
Q 24.90625 48.390625 20.390625 44.140625 
Q 15.875 39.890625 15.1875 32.171875 
z
" id="DejaVuSans-101"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- ocean -->
 
 
 <path d="M 34.28125 27.484375 
Q 23.390625 27.484375 19.1875 25 
Q 14.984375 22.515625 14.984375 16.5 
Q 14.984375 11.71875 18.140625 8.90625 
Q 21.296875 6.109375 26.703125 6.109375 
Q 34.1875 6.109375 38.703125 11.40625 
Q 43.21875 16.703125 43.21875 25.484375 
L 43.21875 27.484375 
z
M 52.203125 31.203125 
L 52.203125 0 
L 43.21875 0 
L 43.21875 8.296875 
Q 40.140625 3.328125 35.546875 0.953125 
Q 30.953125 -1.421875 24.3125 -1.421875 
Q 15.921875 -1.421875 10.953125 3.296875 
Q 6 

<div><h2>Transferring bytes between Scala/Python</h2></div>

<div><h3>ImageApp<br></h3></div><div>There is one more app here using all of our clients built before. This app does the following:</div><div>1. Gets the default client <br></div><div>2. Gets the oid of the userfield folder </div>
3. Writes a folder to GM Data
4. Writes an image to GM Data from the byte stream passed in using the oid of that folder as the parentoid
5. Returns a base64 encoded stream of the image's bytes.
<div>To use the app:</div><div><ol><li>Create a base64 encoded string that represents the 
image we want to upload with the app. <br></li><li>Stream over the string and 
decode it into a stream of bytes. <br></li><li>Pass in that stream to the `run` 
function of the ImageApp and compile the stream into a list that we save to image. This list contains parts of the base64 encoded string that we concatenate and save to the variable `image`<br></li></ol></div>

In [26]:
import fs2.Stream
import fs2.text._
import java.time.LocalDateTime
object ImageApp extends UserFolderClient with WriteToGmDataClient with ReadFromGmDataClient with SSL{
    implicit lazy val ec = ExecutionContext.global
    implicit val ctxShift: ContextShift[IO] = IO.contextShift(ec)
    implicit val timer: Timer[IO] = IO.timer(ec)
    
    val timestamp = LocalDateTime.now.toString

    def run(imageStream: Stream[IO, Byte], folderOid: String) = for{
        client <- defaultClient(sslContext).stream
        imageFolderMetadata <- Stream.eval(writeFolder(folderOid, s"Godzilla - $timestamp", client))
        write <- Stream.eval(writeImageFile(getOid(Right(imageFolderMetadata)), s"Godzilla-$timestamp.png", client, imageStream))
        image <- getImageFile(getOid(Right(write.head)), client)
    } yield image
}
val base64Image = """iVBORw0KGgoAAAANSUhEUgAAAUAAAAClCAAAAAAIBXvzAAAABGdBTUEAAK/INwWK6QAAAAJiS0dEAP+Hj8y/AAAACXBIWXMAAABIAAAASABGyWs+AAAi+0lEQVR42u1dB5PquLLm//+cN3K2yTnnAWbIaQgmJ+frRDZgG5+zO69WdeueLWBs6VPnbrVc0n/jreH6D4J/C4CiwG/Wv2LsBPFfB6AoShwn8L9jCCIv8E6B6HKG+AT+l3GeiuG/A0BBpjzR5C/3hxVN99rtVmtAb1ViYPd0/7vfKhSK5ULhe7oVVWr+K0PgOE78pwEUBZYTzOz3vEt/paOegBtBwAeE4cFcKhWPxJIxr5eEEBgAxA0BnMh81ir9zd1SRVFBVV8tt9+xwqyzW9Drq/UzzIQ+WMRQEN7drrcAFPiX6PE7RmJ2kyQBefyUB4Fh2BPJtFpRDIOANqDLgUbLNHu5JLbTGzZK1c/SWBQ2mpxg+r3tZlHvj1Yb7Zccp71qGveFvifcgbe2hvekoesd6nsKH7PsDThxnooPv1rLVCDohiEIJgPtdiEdC+KQ4UAL25un7ApuDGTLXzR3osDj68+bpCMmcptR62eQ+Zpbg4TjxX8AQIF9tHPCYdGvfi+27Ta7oTejyU+hNqvVQkEM9jY3DNvAPz4gYIgf7C+s7rZpkkjs1p2NsWRgF1Pm+qMDs2mG8z/r1ZoVbw2FJ6Rgm5PtAig8JvxdP5vwovFhNDcbZEOReuvbG65UMulse6FQ7Lrd7Q7rHiMI4UBrbyAFWOP3sN1sqlLp7I/IiRK/GlSiyQgmS1SSIALlS2zZQblc6+0364NoaIaxwt8DUN6wh1+NW53JYj+pl9r9/mK1/RmMorV2MV0o5fPdyyn2UAP8sPSCMz0L/hNOTS7U6J7jmh74KFUBjlGZEccrtoqgvFjcd7++K/P1dMo+WJRoi5Nddpj3kZQWVyUMRlGyuV5vO9X5/Mud/IpHqs18PJHIprOXbDglbuHD3J6vnYVptJGkTmGDLrPl2Hk24oMvdoPMRLyRSCTzM2+kClVapk9JpOs/+8eC25ZVYxlAUYNPZAxeNvaTyRgKIJT0x8mp2PZ784XZKg2HKADcFJRdnpAXCtidAqFK/BNpJd58eohATe0TJgzlu1+VdChIHp+FqEhqWh7BSRwHRJ3fr9eLus8diBeq4/3tK4S18pEgCH8aQFljiYdlv5oJVduzzYHVWY5nDrKHeaB3dMwrq1uAktGNbFeEkZw0DWAEDGMyrjBZnB+XX0xdK2IUT/qq+qoO7J2y2HaKpULrctnivJwaHnbyJxMC4MUwIe+bnzDUTQCRdwuJJykSR1VUIdSTKRRyxT6jO/G8xLQnvEoX3B8FUPYdDtOCF1dEDQyjpDcY/Z4fBo3PeCiYzBU/p0wjTOIwIv8iORrWuj68sOJ/Qv7yVy0hEwXAajoEo0zhgt2UNRHp/gNlcVh+xn0IgCkykf+ZMZoFsxqL/EHY9euF5rhEgXCRxGAIRqDHA0AXNqdGnHCsXiu3P6uNdK5QbU/2wnPt+DaAMoGPYz7sPA91Nwk/okwNgA/YjfkLg/JXDA18d8bDQKBEIr7cIAEX90yvlJXVBtnUXQW66DsvBg3F8q3po61n6yFt6X5U5kc0WJ3xTG247rQSyVIy4gcQForBcKJJuWHI6gDq/2AK08gy+T2WfUlR+EMAirLvSGOGs4B8XoR0ozCE+MPeqEfmYL/PQ5KwzLYJHA7OJa4+rBOQ/H18qjxqG9B3QZm4d8AJugssGqinRRxWf6i+B8HAB9rkS+WeHzo5MvJbkGQftY7f5Qq0dyCeXKe3sGLSmAeQ+fmMpYq48TRRb2Hw6YcgN6KuSeZxglT+xahYbshIHLtJy9Dns3BkpDxreOQ10hOra7Yz12gOKoXk/I6BQ+ryvBFcoTV3sVepfSZCyDWzwriXgpwYSMDrwdHgwLw+NgugKOwTfi/x8M0eX9CrIokpOwkjGtVQmSiWY6RZOh12eyCQnOlybqYTcmSxP3LuoYbBHzJ13bx3m9WsuvnKj2EYjoSWrVbw494KB8ARAOHw4jAuhFuCaZvQZRI+UdzPBpnHQhrgPq8iTqIJDFBuX4iS2SpUH36HPgsjdj7f/3ggL05M9OdtNIPDPbl4x7wUSTYW18zDDYPqD72NUQjzeb0QTHgjb/DqawDJ3FyQ2K2wpDcOAihwPF32udFnYgQLRmWWRcMBwl2QBaEHS3RlJ2vfHqw33Q695NvRYLVzJDc+pojv8Pw2xqkLC/1foacYlfIPQ5utF09WKJlNceKP4AbwkPZfH76yuqvcbis4BqDAc2VVT8nSVuYizIBfYCL09YmjARjzU1EU9iPhIauDsegRsl3zw7HLC0U7cCMfxPiRramzz4rQ39OSGJ8vmwoQ6SQS9zmPX76XCw2DygpBbH1SwqJTAApbehxDVBVIRaKBbC/lpW55mfR6SAiVPyWjuGw2o8VzZFNIK8gHFtzuAjBx/eX5ejJDfqEwultXkOHxpuaLFb9jqTYBo1c0+BY7qySBQMhY4jdbHxJKN7P0BdvJ0xDfB1BQQsCHphdDsWCTHtCy1OcOmzpyFwjAqKhPYQYAUDh4dmqZmsL5IM0uo+nL6Qj7x3FPYbNTvmSCH1icTPshqidwo3z6y4coz/IHLpXXG9oXTjbqlcZPOdiR3zZFvOOtdOPK8a+18UsA1Ufwm0ExEryIBhx8t1xMUgiGQfEYQsjY4tOzGtV+iYVJ4GNMMQX7U04uVRybqd4uF49ig71Ed5MgiMOU7CaGMrI9edxAt98+gPhSd0Nl/MQ6hNVWBrav8BaAumHLLbrVaedref5ifS+JUJQgZZ/UnZS/wtI1DWxe5MIyrpBsFQLgMRdsOVRg9EvfvX5zSkD4eCYWYzHZeTxxrPscN7DPwPEL54ePKkb/+h4BSXgHQJaXRJbufjWW/LVM3UfuolEJXLZlSAJDvYhiloUmslIV5usaFSUV9azIwa05j7GGQHVdjfgybX+8zUpC5OOa+6zKPgMHAG5dvvVb+UGZvw8FvWBj1/OEpbga0pP5vZe//wrd2vBRBJF9SsRLUJRmGIZL/LoavAzRhYevhTL/E0snSI/unsQKddmzGv30Am8ayqn7EDjVoWWRdyQwrizve3Ah2y83qQDZjbIJIL9brATGKO4nzRtj7x0JwrK1Eix6UepoLzakukYtlI428C1fAtglweeyCGKcov5/IlTIkwyhKGpFNxh9WI7e7wAaKSU746MxKjRQEN+rTsN9WtUWgLv1Q6d6SNOG2s/dbgHVmlHVSp+NKQEm1I35CApTzYbmS5+nBrCfDgGhJVb4dqPwfebzNbcaoO2pEcYRLjgcHh1f3SFCJ+tLYC7zFwfO4aQSK04JQ1c8m1XihBoRIE1l1oTiecnOMQrJ7B146R/t07npNivjljy0UIf8WwzCM+6H4YN4anF8+U90fxu/4zZbhSofG4SPAHxROLLNEAacgidCHigYpEgNXRSmEHeIkKUPSVEUHu4WWiZkoCgt07nCdzOMWkXq3ibEFWZAA88irSA1jB65mB8Kt3mEo+4ULAP4IrYVp4IGu4kjOCbz3VlzIKTioGdHo6/h8tAga0sz/tF6y8+6oRfkhyBPeVdLGOBu5FYoIresg7QbIZ0Gxb1TSaXXyxwHs9ckqMZIAOwJyrK5ETvzD4X4o8Hw+LATmzKYGTMph0OpGIDBiwBo8GlUAQ6pATP4vgQCv7VgQXRbXnOv1/0gUO0yTly+Tiri17Fp2KtPlSy2R9KMkn1MFFUED+aVNUJVNhf2zWIykGTNSNh2+CF8qkYh5EeG1A1DdfsO3GphigI4DsG+18EbkBJXHfZV8YKSjjQP4Os6Bzb2aInupD/yXQyQCEKokkdmNQBnhuO6IIk7mjbHF2vvg8fDiSBGNkZHb9jfX5bUXYucGJpMUkqNCAFUJjehhkBGEps9NZ/Evli3aBJAE4Gw5mNPACDFcmAyKJ0yDbDPi0YqI0uSpXD8YwjNUpfsdthEfaM6on2d2UqLjLJJhfJxPsCr2pyEyRiDUiRWkulByWkeFi/YwzAXbs+MoanHWwuSVdy/qSRjEVXa4JF6KfI5NZ1uValfTAI14eSOIvh0fA5cEDIF/zS3kzSmfAR3JSamuHhosYPZC3DhBIAH8gsbucn6dXGhaECi9wCayItun6hI4Jt4gCePx6pdxf1KLzihvzG/Ndx3uvj91XADiMjGknM2DwIrOqBTHEjpxoEwVIIwIMxKA0JTrVewHU351wwMcEw1nrswHF2YyIvf+8UuG/iJhSczA59NGAmUMhSspIvxuUXa3rn1jNFHQuA5trFOfWR3635EEwZdpQprqzgJu7iSgamL4iiA3kJFFDQRCpMvqZEYzJXdFT/T6b6ZED7HvwKQN6EmV0cGJrwGcwrggMiEMU2g161aVpxO3EhciV8fvvh9kfK0uREOAwSHEz/TLy8VbS+WcwVACG2L3KgE37kesJ5neAKgOkHg19l2V5+bS4HckaDLMAD4nABz4JFReuQbLd8dmbCWpevGrRFPiVOKmKbBvSTS4WhnSEC+2jgSj+CKaIRxXDOaALWQhj4bYX3gH3gVIyF9nOF3YGVBRj8B0ISnwPjM+ajhpZkJ3YiMhrY+95pV6vhmOUX5LApbtgACGyalFSccA7iKzt1J26gNl9k7ZHPDnyAyOHNVyhwJ3gZrXFYNGFnnB83MGA0uTLHEmr8i+p5GwXGtjlTgJXE+6Q1EifYRn8yXghmqB6O9FI6CsAxw30ZNTIrmNqJURUndOxd3gZjJeo4bIeey6MLp0e/XU/TSZu2WqwnNwn6ZxuIr2S/d7Veb0YwfRL+Vyr3v0IxdyvYyDsBRyWIEQHNzYYqbwYy8jk1Q5XCVZaY+pKPZZXXcszUrpsVHAIrmzgeYAzBgXf5pUfBZYTRnd18pr9tLUEial3aqImIWvCiMOp/TQnqQuKhTo2IeUxRI4rfmFi1KYlURAjJbfsEQbNrUFx4ByJkk4vzHa/yIpt2y91my0296tdI5oKhJXeiIs+ZSMWSXDSYPrmsLbZUR9VXKw8iJtMynfCgWHpq29fkHAJot6qITL3I6AEnSdvFjZR0Pn2IxcI2/yCSr7sx+LWoAut9KqsNRJfXK9hb7bS4YjqXog/matsuf2nLl2DT2tE7G3eYsP1OP+U6i17B46xutDne7OW3JXrOTKfwdAKlgVMmy7nih4iuW5mtLcoY1AtDKUZPPpx5nfGmD/DQ38xC8EQ8AJoOKnO+5PfWVArLM0LrWQF8CqEwSx4xZHCHQmOpiCgv/t9WzphfJItdjG/vJ+HY3jMMJiuTybSXbo230SPJ7IlRlD4+KlH+GqXI3SZiqC0RkSxGFKRwz3GwURvD6MUVmdZoXUtBlxYc7jVZofx3zBLI7D9zfX2Wf+yNvHz+pZowHlvRraCEouJa//tjDdIciZpQvDRw6oNWCFvZWrA9j899lg4MFaRNtR66WGhou+/mGEnee4Gk7+kOeEcdICz94EojWScdzpUu51CstDN/Ja0SNFlIwMjFv/d7WzNwCKFgS+9tUaXAxLY96xE3bghURsXN6fTvbrhYtrxmTxBvyXAQf+YUPAk8DV/cpORjTTcHNnVI1jeAtgNZ2YRUM7grn+XQvixbwKMtahlBkeZbLQKZMOrhDk2diqnLbIknGbdUY1TTaOdjY8FOKyfU0Y/KYYOKDxSkIjNJXfkqK39lRI2INMevI8l8XyqC7E9fTQRi2blLrHMzYAVA6Bqd1AK02jRCzde5IgiByVff388Nv9za4eOcxu350dLioNfLWJfGrtP1plYu1ZitsHkQQ2E1W9kTgReWB68qMNT9+Cvu9rkdA4zbe9cPtrGuRslnHAoOSQv/MxCDRK6LusS7DZz6zdAg8M1Z23+x6TEfV7bpRKqZHf3E8sopeHFZgf9qNpSBIO8snR43OQBnjF0WoldS8qrFXPtUTL+skeafB7w9GABjzj6RDaC7ZHlcUeORgfi9YUEOClhwm16eMi9j/asaCibkk7CwqEjZulve8MkCh3eo2GK5kJzVn0AtOprNiwRCeWPtau+MI4vmk96Ikjlj7AOo87LrmYGE9Ma/TxRRQZoqocSD+OxZdyea4wK+SgebBKgmal4BqRUKDvU0MYsdolFD1V7CTvQKIsTyV8sWP4fyyWN8Jdi3AMwXyFwCe+wiII/N+DeNTizdAXvnrOfaR0JXJLhftrCe0pXySmLeiQkFRoq9rVuHKSQrxzAI/e0hqNc4hc1EZzYj1huTAuADw0pehJ+YX3SCUcwfAMxfZbRY6JRgYlmeGJBwaWbHOPy0BGOOkCXYVHl0ak7MeDtud4JZV3pIonCTHGy2rNB5TARRsdqwQB0n1MJE7HfTA8tbqn3YG0mJcrXQns6b5+bWtRfPKgpA5n1uGgvSNbNG/yR4trAFyMl5WfO8YvBKyRfsICmcA7Tee4dpBpQpXbQTTvpIOimOn/McrT/14xr5szQrGZtLqSGewN32TAOwcLZyThbrCMerIw2fZIiSJ9Zt62HXmZnvj0I2GqjEChnwdekDfk7J4eOWWq8E1LmTRjaiJYlZrcALSu9vXrkg1AZo553rXiU7nqJujJ0dpFwwzbwpBl71wzhUVstJOtqlxFIWx+7yCuH1B3gd1CUOrBb14g1fEJoCDVQPHsaR8FWSuVrs+Goia1lOnnsy/sXaVcV02/DiDsSz6FHMWHtx/9cqt04I4k4d2NEo9yLtUhwhAAjVDCl+SinLbX71EzAAt1wLIE9/S+WqbMZ6RSSHokl4WlJsb+2ESAdT2LmYrCoyZx88fAoikvA++yAcTo71m/98Rh2Ljn2cjrRU/ZUoWPLAPPVquyth8DvIL4YGbayoi47IYzH8GYY4MsUf4hl1aO7FssosIk36sLwyiNG6lswA1Vfo6MbTA3ZucPeQqNb1RfrHyT7clJSuPnEy1/XzcLv/YdYePFMg61DJy7Elqe8dUw+6CNzxb8zN6azKusI9Y0SJqTwa0sZvWkoGAERPzcQCiZ+Gh2nu79F6W1jCEZo5/waykymdl8E48wWUhH/xqrIPkSD0H3IM/fBkASCodIFNmE/7riOUMOeZFIfCRMhSysiVz0YpLJZZtRVC6WGDdo30vzvbSLPY14GwumNMAFBiHABSZkq/zWSxP/bKS9OvtgaCaWRX1Yz2qrBYi1QyftvEk6NS1fl77lGqEQftEMHPZ/OZas9LE5oIVNeyymI97wYf9SHEboi7KaykYS5iMT2vuFqyWupg3ah5mUSfrxfHY8ZFhM3H+UkWISi+uVS4OFa8CVNai0gqAnGMASuJ6cV3B75GFuVl3aRFXj8gg6tFPFEOeR1WPo/4470AkxCuNySS/L4nloJzsmi/zuN71THwS9jMGkP8jreDvQnVwslKamAHxkIe15iBI4IueV55kzeHj4UFAPq4s5euXHvJWNqf6SPpCYTeJBssz0qy6s0suGgU63bf53qSTUcGKZlI3XM0dqXcb2a5yImMXxB9VrqHIuabd7LQ6wTK/c39UT+9qkcC/4iVbXQP/KIAz45qVjJlZilv2zEub1TxpiCAIHwtLQNA09XBF5JvL/l9GfvhB7TIBA7Svu+s21Si7/xMszFWNF+23MUuWJowi0sDjPyaGpuZjJ7ucpzoig6xKMvOyvM04fZR9B1tEpFCuy6gh81uj8YDt4G8bD5vcqWMsrvRk0Fk4aiEpJPLrNBotRRSzedlTI4agJMoYiHtp2bBtC8sAbnirXrRhikkfQzd4eCrSuohp3qtfD3aKLqQtpcTFeR8F7opS+vmllefB2V271GoMpFHBFoDCTgVw/T7diaNj0Ip/eIwT8rR21t4k8NJdpgTzgmRb2yMQtXiKR1jlZQd5Jk+ipUdlNrvRcjblpMkPN1qtrHb/lPj1DQW+oTiOD3mS3gVEUGkBY0me3rl3OIFM1iqAgJpbneV87gF4gxWkhRbgyfHDTLlIs8JqK/QiRH293J9OXpvhRp0CHVXCs2dlo7JdV92wVqJndwLVg+HtnAZly3r0fJ8HwJ+lWelL8TKxmVSCYpv9fMMwm00vWDzU/Ym+yksHXmSca39nZTCx5yEBggxG0gvzJJi/swBhrYkMsKOTDgvZXfROp8K6nozjnh0bSfqUE7TTdo3eRKg8CQDsUYx+GURu81JCKD6c03Zg51VEQNl6t/kI3OG6YgF49SID4LaXDZpQH2E158VzYxmoznfmU5A2O4bZirNOBFOPKWM5tUxk8VLesAfnAaybiUkBMmP6UAZ9JRNAUAcUlOxJbmHsuQx/tbqTT51VRe7ATBLaw7OH1XiQH3Gbv8/CC3MNJj+ivC09AtzJowtnc9vFRT3XYZgdr95lczic0gH8rslIa83FQaODemUuip3n5w14zZAWOEe1iKkmp8Bt2gWrX/9lQuueCRJ2bQdxHsOjQSrRPx3MUk7yi3tWlKe0UhQ8jIGY5jbtF6tnOXM9GvM4KWfHuhQr5orzTNtwi6sdId1hSi0vitrfdm5O1/OpdH3Y7a768/Fs3N4K3FaVjM1CDCCVWQkfagff+Ply+XimB92VezgVW9mmrZlK8XNq9jWA+FVzKQRVq8ZtudZXxMFsRr1xp7vafRXPFWmCtP0s76V90PezUjmT5aZd4UVEWnq8l7YcnKqJXg9+8zr0cHk8Wa24x4PqaX8n4r+Thbi4OZemhl6XlexE3+IdvXsEhJ4TkRhn41mbl6V+gLByNuiuxQWWViIyWWeEt6Ctfjr42WnmyEyzsVhOv5ZDYJlH1UMO54XPkd6X1c4ZC6TNJu763SptJkBKFJXbTNYjR+bcyYT9mcJG6QiRFS+U7LPCoVNinRecBZB9USoPAlaM4K+77UCTGgUqJCBU42+78iKvOFDySMm0xgVD/NNw05F9mTOAztox8vh+3vYGWDnLdBfekR1qRQ+DuKDZ2e79m7Pdl5OLuurl/HBK4xp3+3564l3Eld+ci4tEp28W5cqocjWHIwBeWzGyr6p3d1T1OD8NQN71m5PNAqBepgG8M1G5aAIQ93mqzcpgyqfyNsnxq1n5UQSGH/YyAGELL+xdiz8SwvWWOUul6I8AoPiWABIXyt1E+uF3qrUXygAyAJA5uSSbsXDmfOlRher7UpFtE49ZOGrh8Velv2hEJz+yoriptQ9gWFFnxfUMKPtx7LWFFHpKy83ePc7HqqkxvTUAUBAcB1AS+xigjAsMHhRjPAgmYLc1RQp+mqlRBLJPuHlrlrXbqiXl/zoPfQqxt7kBSavSd1qLqK8MA4OGxODBBB/babl7SnbrsZyWcsuBeMyf2LABDx3DAHpieaeB78v59XaNWjSG/QMASkYnPzDlhguvJaKh72Lc4JgcLx+P2orC1rJnJ64HKe+DGhzfpWISRrd3KItnDj4e9foTl0obNckjYrBySsYSmaTB3SkRbT0L5cJEtbSI2VqlQHFfIZHHpSNXPDJq7K5ZVMVTN/5cF/LwLwCoHF/zWuzxcJdmAV4tndmWRWDO5tbT4aenu69OISplhAYbK14CyP0VAAkcor5pq01RdreZZuDXRM4EheC+XVv/hb/++jAYdxWR/hMsfH94C4YLa+v6/XAbH0MK2nQX2IfPphvCvfI2/esXkByb77gu/GKHR+tujiBuR1vdNDsD0ePhar6Rn9qc2z7xKmL0KufC8dcA/gEh2Lvf14qtB2XPi8Xgq4fYZpzJy5uEXvHwTc8E053bLIz7xojAVksZqXe6CEjt1lt3YG5r8tVpxhdRstuuHX9CCjbuWfjT1oP4/KldkyKddg7MjfG/YGHiRf/SUxDadeOr/AEKPJ9K9dh0uzYXjbrIqRNze9VE8lXO5ty/zSVJf0oKjjQA0cCRA4kvu2H3i/ZOVUfmdqnZDfrNAM+LGNm5fZu97m3m5IxGNzBxkoC227qNTi5X6uDI3HjdjAHKpZaxyK1HDDdf7OjZbnbdi0WnhpgGZxbG/VDQPvMt9SgWnHYGP2nrAWpDal+lt+AElg6Cy46I6OcrG8aof6DzDvHFIXQYQT8K9p+/JJVrUfFQwyGPiSvLm+ENuMvHSH0Fi5wdRvRVwS9v2MHS8K6Ct0bxoj9TMuDt23+S0El8zxYLx4JGhyQabO+Zc8Vst1M9s3H1Dr/F9cZdBqAvi4tYZz1i9U6LU8vX9eEdEeG0imPn17KA/eke1Qpcvt+n0mOr0GUQdnUuonrhv35J/+YhHhjNuQMfMQM6r3fNXYsmOKpH9uee+7K8rkn/7sEElb7VcCBmoOqETGGye+A/3l5G4Gg45hyHDn3u/uUAiotGvhFvGVVGcw0PXmFvowjGAPJOInisqwLeBSdK//rBMxJjmBjoo+Exf++DGFOgk9mRBakeCkQ8nrX0CwYz2kzvfU1usG4nz46x+OJCFkfVHZvzwuoZrwn/GwCkcaJ8xyjrrKffZ56oibsaad65xQo17USaryH+BgBbZPxWUPPjQHAsPtUR90XmDlLLpqYdyt/9BvykRe/GT9wPE/6cnjPQLsg1KMMyqNJ3zqXjlMa8KJxgpV84uG4AzmyOHKuWIYgmL+ZzLDrNpFEM8Xo3vxE/oQxhnzc7bxZAx+oFd2V3EIXQT6eVsPBXCLA2u6lHEF56Io6PqVo3pl6Y/uso8E4XGBc/PLhf+OCMHKQpkpKlYP/vrJkbHv7cw0UzvvClVe4ImwhZpZrobwlBfrR3CCzRvGJ4xMKcM5u5zWIQSGgcfFj9Fk5mLEz0oQx0qKPWPAwgXC0i5eqFlfRLx5PaL5dlprc4Jj4yGBo/EMuOS/4/Q+PPYvWuP24tzIsBjKgtDYSqQG9+Cf2Jki0AnbK3ls1komMkALn9b8DveZjZ9XfmwP2KcIIx/fHSGwAKv3fhTok/QZL+eQr8vfhxr4JTrjdFwP/vIbxOcZigQIdLB/V67N8gFc0IMDMsLDrXGEUU9jz/W9jX1Hktl7lHOWWg8ofDevk74qu8uUoXl0m2c4aN+eUkExv+ChI0SzMuR+n51UP2BQLuco6x2L2I548O4vp7tVlOp8odBFo6g9d9WtFUSIMzzSUu6S8iuGkTAHGmx4HE0vlIpHrTL4OvNFu50oqmRxGIpAjM01rQBV+iv90tK618i+lnCrlwY7o+HC5KnQxUBWdexVmwA8V3rWqhHoEgKLx5VyWJzKJdiXkAAEgoW222fmgtzMMu66R6pTOqJPS1wl0cVU65U24SRgAWwNRyXpT0BUrb6xbT10rTAq1YMqS5dyoIeXExHSnFHiFT3e9OtXtnV4pfb5oLXhim/fixsFn9B0H9nQN76IVx+EGzhuOvL9oyRzP577lh4N2amWDNExHsKxORrvkI5TZlQK1fEdhht/mJlfvjybDTaeeiua92b7oYNpKkDyGjOdzgkCUcCnlRS5cZKAW+WJje3zbNsdoH1KorJ5gP4gn8tZTvEPrKM88ktLj4DHjdSocxGFFLa1TqgVFCv771wRFVy1e76gcHPL6r7vDWKcS6L8w/0SZXwqMXiURKg0Gn3d8yu91uXv8s6ueDyo+esGxXk27tEoi/MRQq9DcOl9aaZRllI5ggcIYeosBzm3aq1BrTND3bzvrreSOoHNBUGt8H3G43ieBevd4NMz4xzK9ifw07dRqBz2b/Ej47hoataAx/o+ZFftcspSJhSjkDjqAoilFuL+XJZ3zntp8q+x0l1ufyXh2Jyyzx17CDAOL5njJXvgJny8C3Gc5SbnI9WrPLn++YB71mPKAew4AedY6B8WBhMNufKVlk6S8K+nvEh+d+VuIV+9i9JNJ+PFAmeI7fNyIRPwJZ5ztFB/oi+dZkNe7JPF8JYdBfHCDWzxdzhfp4x8qaTpZ83Dut4G0bJodexg29IbSUP0UJFJZZ/i8Sn37wR2UYzB2IRNLdd/r/vRWRFlafPuTNpf9l5G5e/gH5Ppdv+ahvhvTFQz9J/LMgvLN3RLL/bgXG+zkRYfEdxcGvwxAAPNpYvB8gcSSpxM0qQQz6PSDKM8XD1ZkjcTWnsnLs7DtGob9kUInm3KmwuINpTX5D/5KxcTBN9l9e+D8A/9nxP2Pc9ZfbFYf7AAAAU3RFWHRjb21tZW50AEZpbGUgc291cmNlOiBodHRwOi8vY29tbW9ucy53aWtpbWVkaWEub3JnL3dpa2kvRmlsZTpXb3JsZF9tYXBfYmxhbmtfZ210LnBuZx/iayIAAAAldEVYdGRhdGU6Y3JlYXRlADIwMTItMTAtMjBUMjA6NDU6MTIrMDA6MDAJ2VASAAAAJXRFWHRkYXRlOm1vZGlmeQAyMDEyLTEwLTIwVDIwOjQ1OjEyKzAwOjAweITorgAAAEV0RVh0c29mdHdhcmUASW1hZ2VNYWdpY2sgNi42LjktNyAyMDEyLTA5LTIxIFE4IGh0dHA6Ly93d3cuaW1hZ2VtYWdpY2sub3JngbQpBAAAABh0RVh0VGh1bWI6OkRvY3VtZW50OjpQYWdlcwAxp/+7LwAAABl0RVh0VGh1bWI6OkltYWdlOjpoZWlnaHQAMjk0MC3poJQAAAAYdEVYdFRodW1iOjpJbWFnZTo6V2lkdGgANTcwME0ZI+0AAAAZdEVYdFRodW1iOjpNaW1ldHlwZQBpbWFnZS9wbmc/slZOAAAAF3RFWHRUaHVtYjo6TVRpbWUAMTM1MDc2NTkxMl0naoAAAAASdEVYdFRodW1iOjpTaXplADEwOUtCQvSGAXYAAAAzdEVYdFRodW1iOjpVUkkAZmlsZTovLy90bXAvbG9jYWxjb3B5XzUxYWM4YzU0ZjA1Ni0xLnBuZzDTGJ0AAAAASUVORK5CYII="""
val imageStream = Stream.emit(base64Image).covary[IO].through(base64Decode)
val image = ImageApp.run(imageStream, mainFolderOid).compile.toList.unsafeRunSync.mkString("")

### Displaying an image

<div>This cell uses matplotlib to get the image bytes from the base64 encoded string stored in `image` from the previous cell. It then displays the image using the colormap specified.<br></div>

In [28]:
import base64
import io
from PIL import Image
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

base64Image = base64.b64decode(image)
bytes = io.BytesIO(base64Image)
readImage = mpimg.imread(bytes, format='png')
cm = plt.get_cmap('gray')
differentColorImage = cm(readImage)
imgplot = plt.imshow(differentColorImage)
plt.show()
plt.close()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 2021-03-09T11:22:49.105138 
 image/svg+xml 
 
 
 Matplotlib v3.3.4, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
z
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
z
" id="DejaVuSans-48"/>
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 50 -->
 
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-53"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 100 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 150 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 200 -->
 
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
z
" id="DejaVuSans-50"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 250 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 300 -->
 
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
z
" id="

### Changing an image in Python and uploading it to Gm Data

<div>In this cell we take the same image as the previous cell and color map it using the `hot` colormap. We then display the image and save it to a base64 encoded string variable called `newBase64`<br></div>

In [30]:
import base64 
import io
from PIL import Image 
from matplotlib import pyplot as plt 
import matplotlib.image as mpimg

base64Image = base64.b64decode(image) 
bytes = io.BytesIO(base64Image) 
readImage = mpimg.imread(bytes, format='png')

buf = io.BytesIO() 
cm = plt.get_cmap('magma') 
differentColorImage = cm(readImage) 
newImage = Image.fromarray((differentColorImage[:, :, :3] * 255).astype(np.uint8)) 
plt.imshow(newImage) 
plt.show() 
plt.close() 
newImage.save(buf, format='png') 
newBase64 = base64.b64encode(buf.getvalue()).decode("utf-8")

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 2021-03-09T11:35:18.989634 
 image/svg+xml 
 
 
 Matplotlib v3.3.4, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
z
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
z
" id="DejaVuSans-48"/>
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 50 -->
 
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-53"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 100 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 150 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 200 -->
 
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
z
" id="DejaVuSans-50"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 250 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 300 -->
 
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
z
" id="

### Upload New Image<br>

We then can use the ImageApp from before to create a new image stream from `newBase64` and upload it to GM Data in Scala . The resulting base64 encoded stream from Gm Data is then stored in the variable `image2`. On the next cell you can see the image from GM Data being displayed again with matplotlib<br>



In [32]:
val imageStream = Stream.emit(newBase64).covary[IO].through(base64Decode)
val image2 = ImageApp.run(imageStream, mainFolderOid).compile.toList.unsafeRunSync.mkString("")

In [33]:
import base64
import io
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

base64Image = base64.b64decode(image2)
bytes = io.BytesIO(base64Image)
readImage = mpimg.imread(bytes, format='png')
imgplot = plt.imshow(readImage)
plt.show()
plt.close()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 2021-03-09T11:35:32.430192 
 image/svg+xml 
 
 
 Matplotlib v3.3.4, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
z
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
z
" id="DejaVuSans-48"/>
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 50 -->
 
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-53"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 100 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 150 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 200 -->
 
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
z
" id="DejaVuSans-50"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 250 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 300 -->
 
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
z
" id="

# Fun

<div>Since we have all these cool variables built up from the cells above, we may as well use them to make something cool. These last cells do the following:</div><div><ol><li>Takes the last 16 elements of `colorsList`(holds colors from file contents uploaded previously)</li><li>Takes the base 64 string containing the image and turns it into bytes</li><li>Iterates through the list and uses the `Image` library to paste those into the correct positions</li><li>Displays the new image<br></li></ol></div>

In [35]:
val colorList = colorsList.takeRight(16).toArray

In [36]:
import base64
import io
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

base64Image = base64.b64decode(image)
bytes = io.BytesIO(base64Image)
im = mpimg.imread(bytes, format='png')

imageSize = 100
pixelSize = 400

new_im = Image.new('RGB', (pixelSize,pixelSize))
i = 0
j = 0
for color in colorList:  
    cm = plt.get_cmap(color)
    differentColorImage = cm(im)
    newImage = Image.fromarray((differentColorImage[:, :, :3] * 255).astype(np.uint8))
    newImage.thumbnail((imageSize,imageSize))
    new_im.paste(newImage, (i, j))
    j = j + imageSize
    if(j == pixelSize):
        j = 0
        i = i + imageSize  
       

plt.imshow(new_im)
plt.show()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Created with matplotlib (https://matplotlib.org/) -->
 
 
 
 
 
 2021-03-09T11:35:38.946728 
 image/svg+xml 
 
 
 Matplotlib v3.3.4, https://matplotlib.org/ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 0 -->
 
 
 <path d="M 31.78125 66.40625 
Q 24.171875 66.40625 20.328125 58.90625 
Q 16.5 51.421875 16.5 36.375 
Q 16.5 21.390625 20.328125 13.890625 
Q 24.171875 6.390625 31.78125 6.390625 
Q 39.453125 6.390625 43.28125 13.890625 
Q 47.125 21.390625 47.125 36.375 
Q 47.125 51.421875 43.28125 58.90625 
Q 39.453125 66.40625 31.78125 66.40625 
z
M 31.78125 74.21875 
Q 44.046875 74.21875 50.515625 64.515625 
Q 56.984375 54.828125 56.984375 36.375 
Q 56.984375 17.96875 50.515625 8.265625 
Q 44.046875 -1.421875 31.78125 -1.421875 
Q 19.53125 -1.421875 13.0625 8.265625 
Q 6.59375 17.96875 6.59375 36.375 
Q 6.59375 54.828125 13.0625 64.515625 
Q 19.53125 74.21875 31.78125 74.21875 
z
" id="DejaVuSans-48"/>
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 50 -->
 
 
 <path d="M 10.796875 72.90625 
L 49.515625 72.90625 
L 49.515625 64.59375 
L 19.828125 64.59375 
L 19.828125 46.734375 
Q 21.96875 47.46875 24.109375 47.828125 
Q 26.265625 48.1875 28.421875 48.1875 
Q 40.625 48.1875 47.75 41.5 
Q 54.890625 34.8125 54.890625 23.390625 
Q 54.890625 11.625 47.5625 5.09375 
Q 40.234375 -1.421875 26.90625 -1.421875 
Q 22.3125 -1.421875 17.546875 -0.640625 
Q 12.796875 0.140625 7.71875 1.703125 
L 7.71875 11.625 
Q 12.109375 9.234375 16.796875 8.0625 
Q 21.484375 6.890625 26.703125 6.890625 
Q 35.15625 6.890625 40.078125 11.328125 
Q 45.015625 15.765625 45.015625 23.390625 
Q 45.015625 31 40.078125 35.4375 
Q 35.15625 39.890625 26.703125 39.890625 
Q 22.75 39.890625 18.8125 39.015625 
Q 14.890625 38.140625 10.796875 36.28125 
z
" id="DejaVuSans-53"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 100 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 150 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 200 -->
 
 
 <path d="M 19.1875 8.296875 
L 53.609375 8.296875 
L 53.609375 0 
L 7.328125 0 
L 7.328125 8.296875 
Q 12.9375 14.109375 22.625 23.890625 
Q 32.328125 33.6875 34.8125 36.53125 
Q 39.546875 41.84375 41.421875 45.53125 
Q 43.3125 49.21875 43.3125 52.78125 
Q 43.3125 58.59375 39.234375 62.25 
Q 35.15625 65.921875 28.609375 65.921875 
Q 23.96875 65.921875 18.8125 64.3125 
Q 13.671875 62.703125 7.8125 59.421875 
L 7.8125 69.390625 
Q 13.765625 71.78125 18.9375 73 
Q 24.125 74.21875 28.421875 74.21875 
Q 39.75 74.21875 46.484375 68.546875 
Q 53.21875 62.890625 53.21875 53.421875 
Q 53.21875 48.921875 51.53125 44.890625 
Q 49.859375 40.875 45.40625 35.40625 
Q 44.1875 33.984375 37.640625 27.21875 
Q 31.109375 20.453125 19.1875 8.296875 
z
" id="DejaVuSans-50"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 250 -->
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <!-- 300 -->
 
 
 <path d="M 40.578125 39.3125 
Q 47.65625 37.796875 51.625 33 
Q 55.609375 28.21875 55.609375 21.1875 
Q 55.609375 10.40625 48.1875 4.484375 
Q 40.765625 -1.421875 27.09375 -1.421875 
Q 22.515625 -1.421875 17.65625 -0.515625 
Q 12.796875 0.390625 7.625 2.203125 
L 7.625 11.71875 
Q 11.71875 9.328125 16.59375 8.109375 
Q 21.484375 6.890625 26.8125 6.890625 
Q 36.078125 6.890625 40.9375 10.546875 
Q 45.796875 14.203125 45.796875 21.1875 
Q 45.796875 27.640625 41.28125 31.265625 
Q 36.765625 34.90625 28.71875 34.90625 
L 20.21875 34.90625 
L 20.21875 43.015625 
L 29.109375 43.015625 
Q 36.375 43.015625 40.234375 45.921875 
Q 44.09375 48.828125 44.09375 54.296875 
Q 44.09375 59.90625 40.109375 62.90625 
Q 36.140625 65.921875 28.71875 65.921875 
Q 24.65625 65.921875 20.015625 65.03125 
Q 15.375 64.15625 9.8125 62.3125 
L 9.8125 71.09375 
Q 15.4375 72.65625 20.34375 73.4375 
Q 25.25 74.21875 29.59375 74.21875 
Q 40.828125 74.21875 47.359375 69.109375 
Q 53.90625 64.015625 53.90625 55.328125 
Q 53.90625 49.265625 50.4375 45.09375 
Q 46.96875 40.921875 40.578125 39.3125 
z
" id="

In [37]:
import cats.implicits._
object AppendFileApp extends UserFolderClient with WriteToGmDataClient with SSL{
    implicit lazy val ec = ExecutionContext.global
    implicit val ctxShift: ContextShift[IO] = IO.contextShift(ec)
    implicit val timer: Timer[IO] = IO.timer(ec)

    val numberOfFiles = 100
    val random = scala.util.Random
    val timestamp = LocalDateTime.now.toString

    val run = for{
        client <- defaultClient(sslContext).stream
        userFolderMetadata <- Stream.eval(userFolder(client))
        userFolderOid = getOid(userFolderMetadata)
        colorFile = createColorFile(userFolderOid, s"$timestamp - appendableColorFile.txt")
        file <- Stream.eval(colorFile.file.compile.toList)
        firstPart = Stream.emits(file.dropRight(1))
        secondPart = Stream.emit(file.last)

        appendableEither <- writeAppendable(userFolderOid, s"$timestamp - appendableColorFile.txt", client, Some(1), firstPart, colorFile.metadata.head)
        colorFileEither = appendableEither.map(appendable => createColorFile(appendable.head.parentoid, "name"))
        append <- colorFileEither.traverse(colorFile => appendFile(colorFile.metadata.head, secondPart, client))
    } yield append
}
val append = AppendFileApp.run.compile.toList.unsafeRunSync()